# Natural Language Processing with RNNs and Attention

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## Generating Shakespearean Text Using a Character RNN

In 2015 blog post, Andrej Karpathy showed how RNN can be used to train a model to predict the next character in the sequence. We will look at how to build Char-RNN, step by step, starting with creation of dataset.

### Creating the Training Dataset

First let's download all of Shakespere's work, using Keras's handy `get_file()` function and downloading the data from Andrej Karpathy's Char-RNN Project:

In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespere.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [5]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [6]:
len("".join(sorted(set(shakespeare_text.lower()))))

39

Next, we must encode every character as an integer. One option is to create a custom preprocessing layer as we did in Chapter-13. But in this case, it will be simpler to use Keras's `Tokenizer` class. 

First we need to fit a tokenizer to the text: it will find all the characters used in the text and map each of them to a different character ID, from the 1 to the number of distinct characters (it does not start at 0, as we can use that value for masking, as we will see later):

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # char_level=True means every character will be treated as a token
tokenizer.fit_on_texts([shakespeare_text])

We set `char_level=True` to get the character-level encoding rather than the default word-level encoding. Note that tokenizer converts the text to lowercase by default (but we can set `lower=False` if we do not want that). Now the tokenizer can encode a sentence (or a list of sentences) to a list of character IDs and back, and it tells how many distinct characters are there and total number of characters in the text:

In [8]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [9]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [10]:
max_id = len(tokenizer.word_index) # number of distinct characters
max_id

39

In [11]:
dataset_size = sum(tokenizer.word_counts.values()) # total number of characters
dataset_size

1115394

In [12]:
len(shakespeare_text)

1115394

Let's encode the full text so each character is represented by its ID (we subtract 1 to get IDs from 0 to 38, rather than 1 to 39):

In [13]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

Before we continue, we need to split the dataset into training, a validation and a test set. We can't just shuffle all the characters in the text.

### How to split a Sequential Dataset

> Refer notes

Let's take the first 90% of the text for the training set (keeping the rest for the validation set and the test set), and create a `tf.data.Dataset` that will return each character one by one from this set:

In [14]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### Chopping the Sequential Dataset into Multiple Windows

The training set now consists of a single sequence of over a million characters, so we can't just train the neural network directly on it: the RNN would be equivalent to a deep net with over a million layers, and we would have a single (very long) instance to train it. Instead, we will use dataset's `window()` method to convert this long sequence of characters into many smaller windows of text. Every instance in the dataset will be a fairly short substring of the whole text, and the RNN will be unrolled only over the length of these substrings. This is called *truncated backpropogation through time*. 

Let's call the `window()` method to create a dataset of short text windows:

In [15]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

**TIP:**

We can try tuning `n_steps`: it is easier to train RNNs on shorter input sequences, but of course the RNN will not be able to learn any pattern longer than `n_steps`, so don't make it too small.

By default `window()` method creates a nonoverlapping windows, but to get the largest possible training set we use `shift=1` so that the first window contains characters 0 to 100, the second contains 1 to 101, and so on. To ensure that all the windows are exactly 101 characters long (which will allow to create batches without having to do padding), we set `drop_remainder=True` (otherwise the last 100 windows will contain 100 characters, 99 characters, and so on down to 1 character).

The `window()` creates a dataset that contains windows, each of which is also represented as dataset. It is *nested dataset*, analogous to a list of lists. This is useful when we want to transform each window by calling its dataset methods (e.g., to shuffle them or batch them). However, we cannot use a nested dataset directly for training, as our models will expect tensors as input, not datasets. So we must call the `flat_map()` method: it converts nested dataset into *flat dataset* (one that does not contain datasets). Moreover, the `flat_map()` method takes a function as an argument, which allows us to transform each dataset in the nested dataset before flattening. 

In [16]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Notice that we call the `batch(window_length)` on each window: since all windows have exactly the same length, we will get a single tensor for each of them. Now the dataset contains consecutive windows of 101 characters each. Since GD works best when the instances in the training set are independet and indentically distributed , we need to shuffle these windows. Then we can batch the windows and seperate the inputs (the first 100 characters) from the target (the last character):

In [17]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[: ,1:]))

As discussed in Chapter-13, categorical input features should generally be encoded, usually as one-hot vectors or as embeddings. Here we will encode each character using a one-hot vector because they are fairly few distinct characters (only 39):

In [18]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

dataset = dataset.prefetch(1)

In [19]:
for X_batch , Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


2024-03-13 09:37:01.003966: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


That's it! Preparing the dataset was the hardest part. Now let's create the model

### Building and Training the Char-RNN Model

In [21]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="loss", patience=5)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("char-rnn_model.keras",
                                                      monitor="loss",save_best_only=True)

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2,
                    recurrent_dropout=0.2), # 20% dropout on both the inputs (dropout) and hidden state (recurrent_dropout) ... Remove recurrent_dropout in both current and below layers when training model on GPU
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")) # numbers of neurons in dense layer = max_id = 39, because we want to predict next character and since we want to predict the probability of next character we use the "softmax" activation function.
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
h = model.fit(dataset, epochs=15, callbacks=[early_stopping_cb, model_checkpoint_cb])

> Model training and inference is done on GPU and its weights have been saved in "char-rnn_model.keras" file. Access collab notebook from [here](https://colab.research.google.com/drive/1EeHuSc7t1waluhqLljOPufDPcLbTy39u?usp=sharing).

In [ ]:
model = tf.keras.models.load_model("char-rnn_model.keras")

### Using the Char-RNN Model

Now we have a model that can predict the next character in text written by Shakespeare. To feed it some text, we first need to preprocess it like we did earlier, so let's create a function for this:

In [1]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    # print(tf.one_hot(X, max_id))
    return tf.one_hot(X, max_id)

Now let's use the model to predict the next letter in some text:

In [ ]:
X_new = preprocess(["How are yo"])

In [ ]:
Y_pred = np.argmax(model(X_new), axis=-1)
Y_pred

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)[0][-1] # last character

Success! The model guessed right. Now let's use this model to generate new text.

### Generating Fake Shakespearean Text

To generate new text using the Char-RNN model, we could feed it some text, make the model predict the most likely next letter, add it at the end of the text, then give the extended text to the model to guess the next letter, and so on. But in practice, it often leads to same words being repeated over and over again. 

Instead we can pick the next character randomly, with probability equal to the estimated probability using Tensorflow's `tf.random.categorical()` function. This will generate more diverse text. 

The `categorical()` function samples random class indices, given the class log probabilities (logits). To have more control over the diversity of the generated text, we can divide the logits by a number called *temperature*, which we can tweak as we wish: a temperature close to 0 will favour high-probability characters, while a very high tempeature will give all characters an equal probability.

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=5).numpy()

Let's break it down above code!

Imagine you have a bag of colored marbles, and each color represents a different category. Now, let's say you want to randomly pick marbles from this bag, but not all colors are equally likely to be picked. Some colors might be more likely than others.

The `tf.random.categorical` function is like a magical machine that helps you do this. It's like asking the machine to pick marbles from the bag according to certain rules.

Here's how it works:

1. **Input**: You need to tell the machine about the bag of marbles and how likely each color is to be picked. In technical terms, you provide the machine with a list of numbers called "logits". These logits represent the probabilities of each category. For example, if you have three colors (red, green, blue), you might tell the machine that there's a 50% chance of picking red, 40% chance of picking green, and 10% chance of picking blue. But you provide these probabilities in a special way using logits.

2. **Generate Samples**: Once the machine knows about the probabilities, you ask it to pick marbles from the bag. You tell it how many marbles you want it to pick. In technical terms, you specify the number of samples you want.

3. **Output**: After you ask the machine to pick marbles, it gives you a list of numbers back. Each number represents the color of a marble it picked. These numbers are the indices of the categories you defined earlier. For example, if you have three colors and the machine gives you the numbers 0, 1, 0, 2, 1, it means it picked the first color, then the second color, then the first color again, then the third color, and finally the second color again.

So, in simple terms, `tf.random.categorical` is like a machine that randomly picks items (categories) from a list (distribution) based on how likely you tell it each item is to be picked. It then gives you back the items it picked in the form of a list of numbers.

The following `next_char()` function uses above mentioned approach to pick the next character to add to the input text:

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :] # all columns of last row
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)

next_char("how are yo", temperature=1)

Now, let's write a small function that will call next_char() to get the next character and applied it to given text:

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
    text += next_char(text, temperature)
    return text

Now we are ready to generate some text! Let's try with different temperatures

In [ ]:
print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("w", temperature=1))

In [ ]:
print(complete_text("w", temperature=2))

Apparently our Shakespeare model works best at temperature close to 1. To generate more convincing text, we could try using more GRU layers and more neurons per layer, training for longer, and add some regularization (for example, we could set `recurrent_dropout=0.3` in `GRU` layers). Moreover, the model is currently incapable of learning patterns longer than `n_steps`, which is just 100 characters. We could try making this window larger, but it will also make training harder and even LSTM and GRU cells cannot handle very long sequences. 

Alernatively, we could use a stateful RNN.

### Stateful RNN